# Nombre de clics sur les rubriques du header : du 02/02/23 à ajd

**Définition** : 
- On souhaite enlever toutes les visites qui n'ont qu'une seule action (un seul log)

**Formule** : 

## On commence par le chargement des données
on décommente ce dont on a besoin

In [ ]:
import pandas as pd
import time
import json
import math
from tqdm.notebook import tqdm
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime
from src.elasticsearch_connector import ElasticsearchConnector
#from typing import List, Dict
tqdm.pandas()


pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

## Si on veut lire en requêtant Elastic Search

In [ ]:
es_connector = ElasticsearchConnector(env='monolog')

QUERY = {
    "query": {
        "bool": { 
          "must": [
            {
              "range": {
               "logfile": {
                  "gte": "2023-02-01",
                  "lt": "2023-03-01"
               }
              }
            }
          ]
        }
      }
}

In [ ]:
logs = es_connector.execute_query(QUERY, "logs-new")

In [ ]:
logs.shape

In [ ]:
logs.head()

In [ ]:
def filter_df_startwith_url(df: pd.DataFrame, url):
    return df[df['url'].str.match(f'^{url}', na=False)].copy()

In [ ]:
def format_url_and_get_month_and_datetime_columns(df: pd.DataFrame):
    df['url'] = df['url'].str.split('#').str[0].str.split('?').str[0]
    df['month'] = pd.DatetimeIndex(df['logfile']).month
    df['datetime'] = df['timestamp'].apply(datetime.fromtimestamp)
    return df

In [ ]:
logs_formated = format_url_and_get_month_and_datetime_columns(logs)

In [ ]:
#logs_april_contrib = filter_df_startwith_url(logs_april, 'https://code.travail.gouv.fr/contribution/')
#logs_april_contrib_without_duplicates = logs_april_contrib.drop_duplicates(subset=['idVisit', 'url', 'type']).copy()

# Début de l'explo

D'abord on souhaite enlever toutes les visites qui n'ont qu'une seule action (un seul log)

### Préparation des données

In [ ]:
urls_header = [
        'https://code.travail.gouv.fr/outils',
        'https://code.travail.gouv.fr/modeles-de-courriers',
        'https://code.travail.gouv.fr/contribution',
        'https://code.travail.gouv.fr/convention-collective',
        'https://code.travail.gouv.fr/themes',
    ]

logs_copy = logs_formated.copy()

In [ ]:
def cut_first_row_if_in_urls_header(df):
    if df.iloc[0].url in urls_header:
        return df.iloc[1:]
    return df

In [ ]:
visits_list = []

for i in range(1, 29):
    print('2023-02-'+str(i).zfill(2))
    visits_list.append(logs_copy[logs_copy['logfile']=='2023-02-'+str(i).zfill(2)]\
        .groupby(['idVisit'])\
        .filter(lambda g: len(g) > 1)\
        .groupby(['idVisit'])\
        .progress_apply(cut_first_row_if_in_urls_header))

In [ ]:
visits = pd.concat(visits_list)

In [ ]:
visits[
    visits['url'].isin([
        'https://code.travail.gouv.fr/outils',
        'https://code.travail.gouv.fr/modeles-de-courriers',
        'https://code.travail.gouv.fr/contribution',
        'https://code.travail.gouv.fr/convention-collective',
        'https://code.travail.gouv.fr/themes',
    ])
].url.value_counts()

In [ ]:
visits.idVisit.nunique()